## Importing libraries

In [36]:
import numpy as np
from scipy.ndimage.filters import convolve
from skimage.color import rgb2grey

In [4]:
np.set_printoptions(precision=3)
np.set_printoptions(linewidth=np.inf)

## Normalize

In [52]:
def normalize(image) :
    return image/np.max(image)

## Get border pixels

In [53]:
def isBorderPixel(n, m, mask) :
    # function that takes a pixel position (n, m) and a mask as input and returns
    # wether the pixel in that position is a border pixel or not
    
    if mask[n][m] == 0 :
        return False
    
    for i in range(-1, 2) :
        for j in range(-1, 2) :
            if i == 0 and j == 0 or n+i<0 or m+j<0:
                continue
            try :
                if mask[n + i][m + j] == 0 :
                    return True
            except IndexError :
                continue
                
    return False

In [54]:
def getBorderPx(mask) :
    border_pxls = set()
    
    for i in range(mask.shape[0]) :
        for j in range(mask.shape[1]) :
            if isBorderPixel(i, j, mask) :
                border_pxls.add((i, j))
    
    return border_pxls

In [6]:
mask = np.array([[1, 1, 1, 1, 1],
                 [1, 1, 0, 1, 0],
                 [1, 0, 0, 0, 0],
                 [1, 1, 0, 0, 1],
                 [1, 1, 1, 1, 1]
                ])

In [7]:
getBorderPx(mask)

{(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 0),
 (1, 1),
 (1, 3),
 (2, 0),
 (3, 0),
 (3, 1),
 (3, 4),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4)}

## Calculate and Get Max Priority

In [55]:
def patchConfidence(center, confidence, mask, patch_size) :
    
    i, j = center
    offset = patch_size//2
    
    return np.sum(confidence[i - offset : i + offset + 1, j - offset : j + offset + 1]) / patch_size**2

In [9]:
mask = np.array([[1, 1, 1, 1],
                 [1, 0, 0, 0],
                 [0, 0, 0, 0],
                 [0, 0, 0, 0]
                 ])
center = (1, 1)
confidence = np.copy(mask)
patch_size = 3

patchConfidence(center, confidence, mask, patch_size)

0.4444444444444444

In [79]:
def calcNormalMatrix(center, mask):
    
    x_kernel = np.array([[.25, 0, -.25], [.5, 0, -.5], [.25, 0, -.25]])
    y_kernel = np.array([[.25, .5, .25], [0, 0, 0], [-.25, -.5, -.25]])
    
    x_normal = convolve(mask.astype(float), x_kernel)
    y_normal = convolve(mask.astype(float), y_kernel)
    
    normal = np.dstack((x_normal, y_normal))[1, 1, :]
    
    norm = np.sqrt(normal[0]**2 + normal[1]**2)
    if norm == 0 :
        norm = 1

    unit_normal = -normal/norm
    return unit_normal

In [80]:
mask = np.array([[1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1],
                 [1, 0, 0, 0, 1, 1],
                 [1, 1, 0, 0, 1, 1],
                 [1, 0, 0, 0, 1, 1],
                 [1, 1, 1, 1, 1, 1],
                 ])
center = (3, 4)

calcNormalMatrix(center, mask)

array([0.316, 0.949])

In [39]:
def calcGradientMatrix(center, image, mask, patch_size):
    
    # TODO: find a better method to calc the gradient
    height, width = image.shape[:2]

    grey_image = rgb2grey(image)
    grey_image[mask == 1] = None

    gradient = np.nan_to_num(np.array(np.gradient(grey_image)))
    gradient_val = np.sqrt(gradient[0]**2 + gradient[1]**2)
    max_gradient = np.zeros([2])
    
    offset = patch_size//2
    i, j = center
    
    patch_y_gradient = gradient[0][i - offset : i + offset + 1, j - offset : j + offset + 1]
    patch_x_gradient = gradient[1][i - offset : i + offset + 1, j - offset : j + offset + 1]
    patch_gradient_val = gradient_val[i - offset : i + offset + 1, j - offset : j + offset + 1]

    patch_max_pos = np.unravel_index(
        patch_gradient_val.argmax(),
        patch_gradient_val.shape
    )

    max_gradient[0] = patch_y_gradient[patch_max_pos]
    max_gradient[1] = patch_x_gradient[patch_max_pos]

    return max_gradient

In [43]:
image = np.random.rand(6, 6, 3)

print(image)

mask = np.array([[1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1],
                 [1, 1, 0, 0, 1, 1],
                 [1, 1, 0, 0, 1, 1],
                 [1, 0, 0, 0, 1, 1],
                 [1, 1, 1, 1, 1, 1],
                 ])

center = (2, 1)
patch_size = 3

calcGradientMatrix(center, image, mask, patch_size)

[[[0.713 0.417 0.964]
  [0.33  0.869 0.246]
  [0.6   0.428 0.34 ]
  [0.763 0.652 0.024]
  [0.623 0.871 0.389]
  [0.495 0.356 0.201]]

 [[0.522 0.669 0.904]
  [0.956 0.171 0.172]
  [0.464 0.082 0.268]
  [0.848 0.571 0.554]
  [0.322 0.68  0.617]
  [0.032 0.364 0.616]]

 [[0.449 0.416 0.301]
  [0.011 0.328 0.983]
  [0.459 0.638 0.043]
  [0.942 0.766 0.615]
  [0.459 0.643 0.382]
  [0.608 0.901 0.048]]

 [[0.483 0.746 0.417]
  [0.193 0.98  0.077]
  [0.071 0.692 0.748]
  [0.764 0.46  0.325]
  [0.273 0.62  0.032]
  [0.387 0.829 0.36 ]]

 [[0.82  0.645 0.636]
  [0.835 0.02  0.067]
  [0.386 0.952 0.821]
  [0.548 0.107 0.236]
  [0.675 0.909 0.227]
  [0.562 0.859 0.024]]

 [[0.343 0.428 0.701]
  [0.58  0.192 0.418]
  [0.883 0.559 0.864]
  [0.067 0.373 0.711]
  [0.598 0.703 0.43 ]
  [0.448 0.891 0.606]]]


array([0.133, 0.   ])

In [44]:
def patchData(center, image, mask, alpha, patch_size) :
    
    offset = patch_size//2
    i, j = center
    patch_mask = mask[i - offset : i + offset + 1, j - offset : j + offset + 1]
    normal = calcNormalMatrix(center, patch_mask)
    max_gradient = calcGradientMatrix(center, image, mask, patch_size)
    
    data = np.abs(np.dot(normal, max_gradient.T)) / alpha
    
    return data

In [46]:
image = np.random.rand(6, 6, 3)

print(image)

mask = np.array([[1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1],
                 [1, 1, 0, 0, 1, 1],
                 [1, 1, 0, 0, 1, 1],
                 [1, 0, 0, 0, 1, 1],
                 [1, 1, 1, 1, 1, 1],
                 ])

center = (2, 1)
patch_size = 3
alpha = 1

patchData(center, image, mask, alpha, patch_size)

[[[0.666 0.504 0.265]
  [0.454 0.287 0.988]
  [0.317 0.224 0.488]
  [0.489 0.456 0.963]
  [0.001 0.465 0.782]
  [0.175 0.034 0.817]]

 [[0.701 0.398 0.979]
  [0.976 0.635 0.38 ]
  [0.251 0.377 0.429]
  [0.877 0.449 0.706]
  [0.421 0.452 0.459]
  [0.378 0.072 0.908]]

 [[0.441 0.95  0.033]
  [0.322 0.303 0.229]
  [0.28  0.632 0.172]
  [0.416 0.001 0.563]
  [0.516 0.372 0.052]
  [0.035 0.755 0.514]]

 [[0.723 0.156 0.959]
  [0.653 0.883 0.189]
  [0.841 0.434 0.071]
  [0.553 0.862 0.417]
  [0.923 0.277 0.628]
  [0.264 0.141 0.062]]

 [[0.947 0.135 0.366]
  [0.802 0.081 0.006]
  [0.236 0.547 0.495]
  [0.349 0.589 0.202]
  [0.607 0.847 0.483]
  [0.479 0.057 0.31 ]]

 [[0.907 0.219 0.965]
  [0.976 0.002 0.369]
  [0.065 0.74  0.297]
  [0.41  0.446 0.213]
  [0.701 0.79  0.051]
  [0.734 0.866 0.579]]]


0.023524260007935255

In [62]:
def getMaxPriority(border_pxls, confidence, image, mask, alpha, patch_size) :
    
    Pp, Cp = 0, 0
    max_pixel = (0, 0)
    
    for pixel in border_pxls :
        
        n, m = pixel
        offset = patch_size//2
        
        if n - offset < 0 or n + offset + 1 > image.shape[0] or m - offset < 0 or m + offset + 1 > image.shape[1] :
            continue
        
        current_Cp = patchConfidence(pixel, confidence, mask, patch_size)
        current_Dp = patchData(pixel, image, mask, alpha, patch_size)

        current_Pp = current_Cp * current_Dp # Pp to change into matrix
        
        if current_Pp > Pp :
            Pp = current_Pp
            Cp = current_Cp
            max_pixel = pixel
            
    return pixel, Cp

In [76]:
Pp = np.array([[1, 1, 1, 1, 1],
               [1, 3, 1, 4, 1],
               [1, 2, 5, 0, 1],
               [1, 1, 1, 1, 1],
               [1, 1, 1, 1, 1]])

mask = np.array([[1, 1, 1, 1],
                 [1, 0, 0, 0],
                 [0, 0, 0, 0],
                 [0, 0, 0, 0]
                 ])
center = (1, 1)
confidence = np.copy(mask)
patch_size = 3

getMaxPriority(Pp)

TypeError: getMaxPriority() missing 5 required positional arguments: 'confidence', 'image', 'mask', 'alpha', and 'patch_size'

## Get optimal patch

In [56]:
def distance(target_patch, candidate_patch, mask_patch) :
     
#     print((target_patch - candidate_patch) * mask_patch)  
#     print(((target_patch - candidate_patch) * mask_patch) ** 2)
    
    return np.sum(((target_patch - candidate_patch) * mask_patch) ** 2) / np.sum(mask)

In [14]:
target_patch = np.array([[[0.2, 0.3, 0.1], [0.2, 0.3, 0.1], [0.2, 0.3, 0.1]],
                         [[0.2, 0.3, 0.1], [0.2, 0.3, 0.1], [0.2, 0.3, 0.1]],
                         [[0.2, 0.3, 0.1], [0.2, 0.3, 0.1], [0.2, 0.3, 0.1]]
                        ])

candidate_patch = np.array([[[0.1, 0.4, 0.2], [0.1, 0.4, 0.2], [0.1, 0.4, 0.2]],
                           [[0.1, 0.4, 0.2], [0.1, 0.4, 0.2], [0.1, 0.4, 0.2]],
                           [[0.1, 0.4, 0.2], [0.1, 0.4, 0.2], [0.1, 0.4, 0.2]]
                          ])

mask_patch = np.array([[1, 1, 1],
                       [1, 1, 0],
                       [0, 0, 0]])

distance(target_patch, candidate_patch, mask_patch)

0.030000000000000016

In [15]:
target_patch.shape

(3, 3, 3)

In [57]:
def getOptimalPatch(image, mask, target_patch, patch_size, local_radius = None) :
    
    n, m = target_patch
    
    offset = patch_size//2
    
    if local_radius :
        upper_i = min(n + local_radius, image.shape[0] - offset - 1)
        lower_i = max(n - local_radius, offset)
        upper_j = min(m + local_radius, image.shape[1] - offset - 1)
        lower_j = max(m - local_radius, offset)
    else :
        upper_i = image.shape[0] - offset - 1
        lower_i = offset
        upper_j = image.shape[1] - offset - 1
        lower_j = offset
#     print("i : {} - {}, j : {} - {}".format(lower_i, upper_i, lower_j, upper_j))
    
    optimal_patch = (0, 0)
    optimal_distance = 1e9
    
    for i in range(lower_i, upper_i + 1) :
        for j in range(lower_j, upper_j + 1) :
            
#             print(i, "-", j)
            
            if np.any(mask[i - offset : i + offset + 1, j - offset : j + offset + 1] == 0) :
                continue
            
            target_patch = image[n - offset : n + offset + 1, m - offset : m + offset + 1, :]
            
            # if candidate patch in part in mask break
            candidate_patch = image[i - offset : i + offset + 1, j - offset : j + offset + 1, :]            
            mask_patch = mask[n - offset : n + offset + 1, m - offset : m + offset + 1]
            
#             print("target patch : \n", target_patch)            
#             print("candidate patch : \n\n", candidate_patch)            
#             print("mask patch : \n", mask_patch)
            
            current_distance = distance(target_patch, candidate_patch, mask_patch)
            
#             print("current distance : {}".format(current_distance))
            
            if current_distance < optimal_distance :
                optimal_patch = (i, j)
                optimal_distance = current_distance
    
    return optimal_patch

In [17]:
np.array([[[1, 1, 1], [1, 1, 1]],
        [[1, 1, 1], [1, 1, 1]],
        [[1, 1, 1], [1, 1, 1]],
        [[1, 1, 1], [1, 1, 1]]]).shape

(4, 2, 3)

In [18]:
image = np.random.rand(10, 10, 3)

print("image \n", image)

mask = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                 [1, 1, 0, 0, 1, 1, 1, 1, 1, 1],
                 [1, 1, 0, 0, 1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                ])

target_patch = (2, 1)

patch_size = 3

getOptimalPatch(image, mask, target_patch, patch_size, local_radius = None)

image 
 [[[0.204 0.506 0.174]
  [0.088 0.119 0.99 ]
  [0.101 0.323 0.072]
  [0.171 0.249 0.042]
  [0.361 0.093 0.448]
  [0.828 0.208 0.228]
  [0.502 0.422 0.526]
  [0.908 0.331 0.536]
  [0.797 0.367 0.777]
  [0.531 0.878 0.642]]

 [[0.741 0.826 0.532]
  [0.559 0.162 0.775]
  [0.401 0.943 0.153]
  [0.213 0.573 0.03 ]
  [0.674 0.557 0.482]
  [0.148 0.504 0.632]
  [0.502 0.703 0.942]
  [0.262 0.691 0.609]
  [0.877 0.964 0.434]
  [0.156 0.096 0.919]]

 [[0.204 0.782 0.673]
  [0.329 0.179 0.291]
  [0.771 0.451 0.02 ]
  [0.77  0.859 0.207]
  [0.957 0.594 0.402]
  [0.483 0.608 0.536]
  [0.414 0.224 0.598]
  [0.989 0.564 0.718]
  [0.76  0.556 0.462]
  [0.168 0.84  0.79 ]]

 [[0.311 0.355 0.615]
  [0.066 0.548 0.712]
  [0.763 0.048 0.804]
  [0.487 0.58  0.344]
  [0.919 0.825 0.2  ]
  [0.394 0.618 0.958]
  [0.412 0.86  0.662]
  [0.4   0.079 0.026]
  [0.519 0.728 0.465]
  [0.549 0.952 0.702]]

 [[0.531 0.014 0.33 ]
  [0.719 0.787 0.563]
  [0.201 0.585 0.027]
  [0.069 0.332 0.604]
  [0.333 0.288 0

(8, 8)

In [19]:
getOptimalPatch(image, mask, target_patch, patch_size, local_radius = 5)

1 - 1
1 - 2
1 - 3
1 - 4
1 - 5
1 - 6
2 - 1
2 - 2
2 - 3
2 - 4
2 - 5
2 - 6
3 - 1
3 - 2
3 - 3
3 - 4
3 - 5
3 - 6
4 - 1
4 - 2
4 - 3
4 - 4
4 - 5
4 - 6
5 - 1
5 - 2
5 - 3
5 - 4
5 - 5
5 - 6
6 - 1
6 - 2
6 - 3
6 - 4
6 - 5
6 - 6
7 - 1
7 - 2
7 - 3
7 - 4
7 - 5
7 - 6


(2, 6)

## Update confidence

In [58]:
def updateConfidence(confidence, Cp, target_patch, mask, patch_size) :   
    
    i, j = target_patch
    offset = patch_size//2

    confidence[i - offset : i + offset + 1, j - offset : j + offset + 1] = confidence[i - offset : i + offset + 1, j - offset : j + offset + 1] + (1 - mask[i - offset : i + offset + 1, j - offset : j + offset + 1]) * Cp
    
    return confidence

## Fill patch

In [59]:
def fillPatch(image, mask, target_patch, opt_patch, patch_size) :
    
    n, m = target_patch
    i, j = opt_patch
    offset = patch_size//2
    
    un, dn, lm, rm = n - offset, n + offset + 1, m - offset, m + offset + 1
    ui, di, lj, rj = i - offset, i + offset + 1, j - offset, j + offset + 1
    
#     mask = np.expand_dims(mask, axis=2)
#     print("image * mask \n", image * mask)
    
    mask_patch = mask[un: dn, lm: rm]
    mask_patch = np.expand_dims(mask_patch, axis=2)
    
#     print("image[un: dn, lm: rm, :] * mask_patch \n", image[un: dn, lm: rm, :] * mask_patch)
#     print("image[ui: di, lj: rj, :] * (1 - mask_patch) \n", image[ui: di, lj: rj, :] * (1 - mask_patch))
    
    image[un: dn, lm: rm, :] = image[un: dn, lm: rm, :] * mask_patch + image[ui: di, lj: rj, :] * (1 - mask_patch)
    
    mask[un: dn, lm: rm] = 1
    
    return image, mask

In [23]:
image = np.random.rand(6, 6, 3)

print("image \n", image)

mask = np.array([[1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 0, 1, 1],
                 [1, 1, 0, 0, 1, 1],
                 [1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1]
                ])


target_patch = (3, 1)
opt_patch = (1, 1)
patch_size = 3

fillPatch(image, mask, target_patch, opt_patch, patch_size)

image 
 [[[0.172 0.251 0.798]
  [0.418 0.475 0.682]
  [0.334 0.765 0.592]
  [0.171 0.217 0.542]
  [0.195 0.666 0.118]
  [0.775 0.766 0.284]]

 [[0.662 0.713 0.894]
  [0.607 0.473 0.872]
  [0.722 0.724 0.235]
  [0.254 0.477 0.539]
  [0.525 0.166 0.012]
  [0.681 0.935 0.477]]

 [[0.958 0.396 0.594]
  [0.446 0.673 0.139]
  [0.584 0.967 0.064]
  [0.038 0.756 0.828]
  [0.171 0.694 0.831]
  [0.776 0.889 0.032]]

 [[0.16  0.884 0.316]
  [0.069 0.29  0.745]
  [0.538 0.941 0.904]
  [0.144 0.497 0.264]
  [0.55  0.402 0.647]
  [0.067 0.285 0.143]]

 [[0.231 0.123 0.683]
  [0.443 0.508 0.398]
  [0.469 0.906 0.581]
  [0.142 0.793 0.397]
  [0.93  0.85  0.122]
  [0.271 0.455 0.908]]

 [[0.235 0.046 0.46 ]
  [0.473 0.873 0.704]
  [0.324 0.528 0.753]
  [0.626 0.769 0.058]
  [0.148 0.013 0.856]
  [0.986 0.291 0.985]]]


(array([[[0.172, 0.251, 0.798],
         [0.418, 0.475, 0.682],
         [0.334, 0.765, 0.592],
         [0.171, 0.217, 0.542],
         [0.195, 0.666, 0.118],
         [0.775, 0.766, 0.284]],
 
        [[0.662, 0.713, 0.894],
         [0.607, 0.473, 0.872],
         [0.722, 0.724, 0.235],
         [0.254, 0.477, 0.539],
         [0.525, 0.166, 0.012],
         [0.681, 0.935, 0.477]],
 
        [[0.958, 0.396, 0.594],
         [0.446, 0.673, 0.139],
         [0.584, 0.967, 0.064],
         [0.038, 0.756, 0.828],
         [0.171, 0.694, 0.831],
         [0.776, 0.889, 0.032]],
 
        [[0.16 , 0.884, 0.316],
         [0.069, 0.29 , 0.745],
         [0.722, 0.724, 0.235],
         [0.144, 0.497, 0.264],
         [0.55 , 0.402, 0.647],
         [0.067, 0.285, 0.143]],
 
        [[0.231, 0.123, 0.683],
         [0.443, 0.508, 0.398],
         [0.469, 0.906, 0.581],
         [0.142, 0.793, 0.397],
         [0.93 , 0.85 , 0.122],
         [0.271, 0.455, 0.908]],
 
        [[0.235, 0.046, 0

## Inpainting

In [63]:
def inpaint(image, mask, patch_size=9, alpha=1, local_radius=500) :
    
    # assert patch_size is an odd number
    assert(patch_size%2 == 1)
    
    confidence = np.copy(mask)
    image = normalize(image) # * np.expand_dims(mask, axis=2)
    
    start_zeros = np.sum((1 - mask))

    # change to identify border, then calculate priorities
    while True :

        border_pxls = getBorderPx(mask)
        if len(border_pxls) == 0 :
            break
            
        target_patch, Cp = getMaxPriority(border_pxls, confidence, image, mask, alpha, patch_size)

        opt_patch = getOptimalPatch(image, mask, target_patch, patch_size, local_radius)

        confidence = updateConfidence(confidence, Cp, target_patch, mask, patch_size)

        image, mask = fillPatch(image, mask, target_patch, opt_patch, patch_size)
        
        print("Almost there ! ===> {:.1f}/{}".format((1 - np.sum((1 - mask)) / start_zeros) * 100, 100), sep='\n')
        
    return image

In [78]:
image = np.random.rand(10, 10, 3)

mask = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                 [1, 1, 0, 0, 1, 1, 1, 1, 1, 1],
                 [1, 1, 0, 0, 0, 1, 1, 1, 1, 1],
                 [1, 1, 1, 0, 0, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                ])

patch_size = 3

inpaint(image, mask, 3)

Almost there ! ===> 28.6/100
Almost there ! ===> 42.9/100
Almost there ! ===> 57.1/100
Almost there ! ===> 85.7/100
Almost there ! ===> 100.0/100


array([[[0.036, 0.967, 0.861],
        [0.476, 0.124, 0.154],
        [0.045, 0.268, 0.368],
        [0.993, 0.139, 0.307],
        [0.453, 0.074, 0.414],
        [0.575, 0.961, 0.073],
        [0.573, 0.013, 0.988],
        [0.885, 0.148, 0.807],
        [0.783, 0.512, 0.319],
        [0.827, 0.4  , 0.844]],

       [[0.41 , 0.374, 0.999],
        [0.184, 0.204, 0.408],
        [0.707, 0.769, 0.221],
        [0.261, 0.076, 0.796],
        [0.512, 0.625, 0.137],
        [0.58 , 0.767, 0.906],
        [0.405, 0.697, 0.756],
        [0.422, 0.924, 0.15 ],
        [0.857, 0.648, 0.884],
        [0.755, 0.707, 0.103]],

       [[0.426, 0.886, 1.   ],
        [0.382, 0.966, 0.284],
        [0.343, 0.49 , 0.927],
        [0.512, 0.625, 0.137],
        [0.14 , 0.935, 0.85 ],
        [0.874, 0.871, 0.293],
        [0.228, 0.487, 0.408],
        [0.868, 0.422, 0.208],
        [0.912, 0.145, 0.161],
        [0.467, 0.997, 0.362]],

       [[0.081, 0.94 , 0.57 ],
        [0.645, 0.868, 0.87 ],
  